In [305]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.stats import rankdata
from dtaidistance import dtw
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from collections import defaultdict
from sklearn.neighbors import KernelDensity

In [ ]:
def create_time_index(timeend: np.datetime64, window_size: int) -> pd.DatetimeIndex:
    freq = '10min'
    return pd.date_range(end=timeend, periods=window_size, freq=freq)

def safe_nansum(arr, axis=None):
    result = np.nansum(arr, axis=axis)
    # 判斷原始陣列沿 axis 全部是 NaN 的位置
    all_nan = np.isnan(arr).all(axis=axis)
    
    # 將那些位置改為 np.nan
    if np.isscalar(result):
        return np.nan if all_nan else result
    result = result.astype('float64')  # 確保能裝 NaN
    result[all_nan] = np.nan
    return result

def generate_valid_subsequences(arr, min_len=1, max_len=None):
    max_len = len(arr) if max_len is None else max_len
    
    subsequences = []
    N = len(arr)
    
    for i in range(N):
        for j in range(i + min_len, min(i + max_len, N) + 1):
            subseq = arr[i:j]
            if not np.isnan(subseq[0]) and not np.isnan(subseq[-1]):
                subsequences.append(subseq)
    
    return subsequences

def expand_to_all_right_shifts(seq, target_len=24):
    seq = np.asarray(seq)
    L = len(seq)
    assert L <= target_len, "序列長度不能超過 24"
    expanded_versions = []
    for shift in range(target_len - L + 1):
        left = np.full(shift, np.nan)
        right = np.full(target_len - L - shift, np.nan)
        padded = np.concatenate([left, seq, right])
        expanded_versions.append(padded)
    return expanded_versions

def expand_all_sequences(seq_list, target_len=24):
    all_expanded = []
    for seq in seq_list:
        expanded = expand_to_all_right_shifts(seq, target_len)
        all_expanded.extend(expanded)
    return np.array(all_expanded)  # shape: (total, 24)

def fast_dtw_distance(ref, compare_arr):
    if len(compare_arr.shape) == 2:
        distances = np.empty(compare_arr.shape[0])
        for i in range(compare_arr.shape[0]):
            mask = ~np.isnan(compare_arr[i])
            distances[i] = dtw.distance(ref[mask], compare_arr[i][mask])
    else:
        mask = ~np.isnan(compare_arr)
        distances = dtw.distance(ref[mask], compare_arr[mask])
    return distances
def kde_logpdf_weighted(X_query, X_sample, weights, bandwidth):
    """
    使用加權 KDE（Gaussian）估計 log pdf
    """
    d2 = cdist(X_query, X_sample, metric='sqeuclidean')  # shape (Q, N)
    kernel_vals = np.exp(-d2 / (2 * bandwidth**2))
    weighted_kernels = kernel_vals * weights[None, :]  # broadcasting
    density = np.sum(weighted_kernels, axis=1) + 1e-12  # 避免除 0
    norm = (1 / (np.sqrt(2 * np.pi) * bandwidth))**X_sample.shape[1]
    return np.log(norm * density)

def compute_posterior_weights_from_partial_subseq(
    X_joint,               # (N, t): joint samples
    w_prior,               # (N, ): prior weights
    X_marginal_obs,        # (M, m): observed marginal with NaNs allowed
    w_obs,                 # (M, ): obs weights
    # observed_dims,         # (k, ): indices of observed dimensions (e.g., [0,2])
    bandwidth=0.5
):
    # 驗證
    N, t = X_joint.shape
    M, m = X_marginal_obs.shape
    observed_dims = np.where(~np.isnan(X_marginal_obs[0]))[0]
    assert len(w_prior) == N, "w_prior 長度錯誤"
    assert len(w_obs) == M, "w_obs 長度錯誤"

    # Normalize weights
    w_prior = w_prior / np.sum(w_prior)
    w_obs = w_obs / np.sum(w_obs)

    # 萃取有效維度
    X_joint_sub = X_joint[:, observed_dims]                # shape (N, k)
    X_obs_sub = X_marginal_obs[:, observed_dims]           # shape (M, k)

    # KDE
    log_p_obs = kde_logpdf_weighted(X_joint_sub, X_obs_sub, w_obs, bandwidth)
    log_p_prior = kde_logpdf_weighted(X_joint_sub, X_joint_sub, w_prior, bandwidth)

    # 計算後驗權重
    log_w_post = log_p_obs - log_p_prior + np.log(w_prior + 1e-10)
    w_post = np.exp(log_w_post)
    w_post /= np.sum(w_post)

    return w_post


In [307]:
a_tdf = pd.DataFrame()
for i in range(0,40):
    try:
        df = pd.read_csv(f'../data/Raw_Data/Gogoro/台北市大安區_臺大二活停車場站A ({i:02d}).csv',index_col=0)
        df.index = pd.to_datetime(df.index)
        df.index = df.index.floor('min')
        df = df[~df.index.duplicated()]
        a_tdf = pd.concat([a_tdf,df])
    except:
        continue
a_tdf = a_tdf[~a_tdf.index.duplicated()]
a_tdf.sort_index(inplace=True)

b_tdf = pd.DataFrame()
for i in range(0,37):
    try:
        df = pd.read_csv(f'../data/Raw_Data/Gogoro/台北市大安區_臺大二活停車場站B ({i:02d}).csv',index_col=0)
        df.index = pd.to_datetime(df.index)
        df.index = df.index.floor('min')
        df = df[~df.index.duplicated()]
        b_tdf = pd.concat([b_tdf,df])
    except:
        continue
b_tdf = b_tdf[~b_tdf.index.duplicated()]
b_tdf.sort_index(inplace=True)

tdf = pd.concat([a_tdf,b_tdf],axis=1).dropna().sum(axis=1)

In [308]:
# 假設 df 已經處理好 index 是 datetime 且只保留到分鐘
start = tdf.index.min()
end = tdf.index.max()

# 產生每分鐘的完整時間序列
full_index = pd.date_range(start=start, end=end, freq='1min')

# 將原始 df 補上缺的時間，空值保持為 NaN
tdf_filled = tdf.reindex(full_index)

tdf_filled = tdf_filled.resample('h').mean()
tdf_filled.name = 'raw_data'

In [309]:
window_size = 24
subseqs = generate_valid_subsequences(tdf_filled.values, min_len=2, max_len=window_size)

In [310]:
subseqs = expand_all_sequences(subseqs,target_len=window_size)

In [311]:
# 計算 mean 與 std
mean = np.nanmean(subseqs, axis=1, keepdims=True)
std = np.nanstd(subseqs, axis=1, keepdims=True)

# 建立空白陣列
z_scaled = np.full_like(subseqs, np.nan)

# 標準差 ≠ 0 的 row
valid_rows = ((std != 0) & ~np.isnan(std))[:, 0]
z_scaled[valid_rows] = (subseqs[valid_rows] - mean[valid_rows]) / std[valid_rows]

# 標準差 = 0 的 row
const_rows = ((std == 0) & ~np.isnan(std))[:, 0]
z_scaled[const_rows] = np.where(
    ~np.isnan(subseqs[const_rows]),
    0.0,
    np.nan
)

# 回存
subseqs = z_scaled

In [312]:
grouped_samples = defaultdict(list)

for i,row in enumerate(subseqs):
    not_nan_indices = np.where(~np.isnan(row))[0]
    if len(not_nan_indices) == 0:
        continue  # 跳過全 NaN 的 row

    # 觀測的位置（相對於 trimmed）
    observed = tuple(np.where(~np.isnan(row))[0])
    grouped_samples[observed].append(row)
    
sorted_items = sorted(grouped_samples.items(), key=lambda x: len(x[0]), reverse=True)
grouped_samples = dict(sorted_items)

In [340]:
# 前 3 小時上升
up = np.linspace(10, 21, 3)  # 從 10 上升到 21，共 3 點

# 後 21 小時下降（從 21 下降到 0）
down = np.linspace(21, 0, 24 - 3)

# 合併成一條完整曲線
curve = np.concatenate([up, down])

In [345]:
seed = np.random.normal(loc=46, scale=40, size=window_size)
seed = np.linspace(0, 10, window_size)
seed = curve*3

In [346]:
def normalize_seed(seed):
    mean_val = np.mean(seed)
    std_val = np.std(seed)

    if np.allclose(seed, seed[0]):
        # 常數列 → mean scaling
        if mean_val != 0:
            normal = seed / mean_val
        else:
            normal = np.zeros_like(seed)
    else:
        # 一般情況 → Z-score
        if std_val != 0:
            normal = (seed - mean_val) / std_val
        else:
            normal = np.zeros_like(seed)  # 極端情況保險處理

    return mean_val, std_val, normal

In [347]:
mean_seed,std_seed,normal_seed = normalize_seed(seed)

In [348]:
key = list(grouped_samples.keys())[0]

p_weighted = defaultdict(float)
history = np.array(grouped_samples[key])

print(pd.DataFrame(history).dropna(axis=1).shape)

distances = fast_dtw_distance(normal_seed, history)
weights = 1/distances**2
for row, weight in zip(history, weights):
    key = tuple(row[~np.isnan(row)])  # 觀測的變數組合
    p_weighted[key] += weight
total = sum(p_weighted.values())
for k in p_weighted:
    p_weighted[k] /= total

X_joint = history
w_prior = np.array(list(p_weighted.values()))
w_post = w_prior

(406, 24)


In [349]:
for i in range(1, len(list(grouped_samples.keys()))):
    key = list(grouped_samples.keys())[i]

    p_weighted = defaultdict(float)
    history = np.array(grouped_samples[key])
    if history.shape[0]<30:
        continue

    distances = fast_dtw_distance(normal_seed, history)
    weights = 1/distances**2
    total = sum(weights)
    weights /= total
    X_marginal_obs = history
    w_obs = weights
    w_post = compute_posterior_weights_from_partial_subseq(
        X_joint, w_post,
        X_marginal_obs, w_obs,
        bandwidth=0.5
    )

In [350]:
# 原始資料與權重
X = X_joint
weights = w_post
weights = weights / np.sum(weights)  # normalize

# 建立 KDE 分布
kde = KernelDensity(kernel='gaussian', bandwidth=0.5)  # bandwidth 可調整
kde.fit(X, sample_weight=weights)

# 生成新的資料
new_samples = kde.sample(n_samples=100, random_state=42)
new_samples = new_samples*std_seed + mean_seed

In [2]:
path = '../data/Raw_Data/Gogoro/台北市大安區_臺大二活停車場站A ({:02d}).csv'.format(0)

In [4]:
import pandas as pd
pd.read_csv(path, index_col=0)

,batt_num
time,
2023-12-28 03:17:16,24
2023-12-28 03:37:16,24
2023-12-28 03:47:16,24
2023-12-28 04:07:16,24
2023-12-28 04:17:16,24
...,...
2023-01-04 17:39:03,30
2023-01-04 17:39:03,30
2023-01-04 18:40:24,22
